<a href="https://colab.research.google.com/github/manchann/secure_project2/blob/main/dbscan_clustering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- **이상값 가정 1**
    
    정상값들은 하나 또는 몇 개의 군집에 모여 있고, 이상값은 군집에 속하지 않는다.
    
    데이터에서 군집을 찾아낸 후 제거한 뒤 남아있는 데이터를 이상값으로 처리**.**
    
    DBSCAN ( [wikipedia 설명](https://en.wikipedia.org/wiki/DBSCAN) , [scikit-learn 의 DBSCAN 알고리즘](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html) )ROCK ( [Guha et al., 2000](http://www.facweb.iitkgp.ac.in/~shamik/autumn2012/dwdm/papers/ROCK%20A%20Robust%20Clustering%20Algorithm%20for%20Categorical%20Attributes%20(2000)guha00rock.pdf) )SNN 군집화 ( [ML wiki 설명](http://mlwiki.org/index.php/SNN_Clustering) )
    
- **이상값 가정 2**
    
    **군집의 중심(centroid) 중 가장 가까운 것과의 거리가 짧으면 정상값, 길면 이상값이다.**
    
    **군집화를 하고 데이터가 포함된 군집의 중심과 데이터 개체 사이의 거리를 “이상 score” 로 두고 이용.**
    
    K-means ( [wikipedia 설명](https://ko.wikipedia.org/wiki/K-%ED%8F%89%EA%B7%A0_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98) )EM 알고리즘 ( [wikipedia 설명](https://ko.wikipedia.org/wiki/%EA%B8%B0%EB%8C%93%EA%B0%92_%EC%B5%9C%EB%8C%80%ED%99%94_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98) )
    
- **이상값 가정 3**
    
    **정상값은 크거나 조밀한 군집에, 이상값은 작거나 sparse 한 군집에 속한다.**
    
    데이터 개체가 속한 군집의 크기나 밀도가 “이상” 여부를 판단.

## 이상값 가정 1 : DBSCAN 알고리즘 적용
밀도 방식의 클러스터링을 사용하는 DBSCAN(Density-based spatial clustering of applications with noise)

앞에서 설명한 K Means나 Hierarchical 클러스터링의 경우 군집간의 거리를 이용하여 클러스터링을 하는 방법인데, 밀도 기반의 클러스터링은 점이 세밀하게 몰려 있어서 밀도가 높은 부분을 클러스터링 하는 방식이다. 쉽게 설명하면, 어느점을 기준으로 반경 x내에 점이 n개 이상 있으면 하나의 군집으로 인식하는 방식이다.



In [2]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot  as plt
import seaborn as sns


In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset = pd.read_csv("/content/drive/MyDrive/BigData/04_hashed.csv")

dataset.columns
dataset.head()

,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,2.021041e+13,154.58.159.102,103.177.12.42,6,52897,445,2,None,US
1,2.021041e+13,154.58.159.20,125.66.92.196,6,60579,445,2,None,DE
2,2.021041e+13,154.58.159.164,117.121.178.223,6,63831,445,2,None,US
3,2.021041e+13,154.58.159.165,205.34.95.97,6,55241,445,2,None,US
4,2.021041e+13,154.58.159.102,93.56.164.131,6,52898,445,2,None,US


In [13]:
# def select_unique_Rdate(dataset):
#   time_list=[]
#   for i in range(len(dataset['Rdate'])):
#     time_list.append(dataset.iloc[i]['Rdate'])
#   unique_time = set(time_list)
#   unique_time = sorted(unique_time)
#   print(len(unique_time)) # 24218
#   return unique_time 
start_time = 20210411000000
get_days = 1000000

dataset_origin = dataset

dataset = dataset[dataset['Rdate'] < 20210411000000 ]

dataset.tail()

dataset.loc[2861154]['Rdate']


20210410235916.727

In [22]:
# feature = dataset[ ['src_ip', 'dst_ip', 'Proto', 'src_port', 'dst_port', 'Action',
#        'src_country', 'dst_country'] ]

feature = dataset[['Rdate','src_ip', 'dst_ip', 'src_port', 'dst_port', 'Action']]

In [23]:
# IPv4 전처리
def transform_ip(ip): 
  groups = ip.split(".") 
  equalize_group_length = "".join( map( lambda group: group.zfill(3), groups )) 
  return equalize_group_length 

from sklearn.preprocessing import LabelEncoder

# Feature 전처리
def preprocess_df(df):
  
  # IPv4 전처리
  df['src_ip'] = df.src_ip.apply(lambda ip : transform_ip(ip))
  df['dst_ip'] = df.dst_ip.apply(lambda ip : transform_ip(ip))

  # country 전처리
  label_encoder = LabelEncoder()
  df['src_country'] = label_encoder.fit_transform(df['src_country'])
  df['dst_country'] = label_encoder.fit_transform(df['dst_country'])

  return df

In [25]:
dataset = preprocess_df(dataset)

DBSCAN의  매개변수들

- DBSCAN의 주 매개변수
    - **min_samples** : 핵심 포인트를 중심점으로 간주하는 주변 지역의 표본 수
    - **eps** : 핵심 포인트를 중심으로 측정되는 유클리디언 거리값
- **밀집지역(dense region)** : 특성 공간에서 (거리가 가까워서) 데이터가 붐비는 지역
- **핵심 샘플(or 핵심 포인트)** : eps거리 안에 데이터가 지정한 min_samples개수를 만족시키는 밀집지역에 있는 데이터 포인트
- **잡음(noise)** : eps거리 안에 들어오는 포인트 수가 지정한 min_sample보다 적을 경우 어디에도 속하지 않는 잡음으로 레이블됨

DBSCAN 작동 방식 

- **STEP 1** : 특성 공간에서 데이터가 붐비는 밀집지역을 찾고, 그 범위안에서 핵심 샘플이될 포인트를 지정한다.
- **STEP 2** : 어느 데이터 포인트에서 eps거리 안에 데이터가 min_samples개수 만큼 들어 있으면 이 데이터 포인트를 핵심 샘플로 지정한다. 이 경우 해당 데이터 포인트는 새로운 클러스터 레이블로 할당된다. (min_samples개수를 충족시키지 않으면 잡음으로 분류)
- **STEP 3** : 새롭게 할당된 핵심 샘플로 부터 eps거리 안의 포인트가 만약 어떤 클러스터에도 할당되지 않았다면 해당 클러스터 레이블로 할당시킨다. (만약 핵심 샘플이면 그 포인트의 이웃을 차례로 방문)
- **STEP 4** : STEP 1~3을 반복하며 eps거리 안에 더 이상 핵심 샘플이 없을 때까지 자란다.

In [19]:
# #############################################################################
# Compute DBSCAN
def dbscan(feature_):
  from sklearn.preprocessing import StandardScaler
  feature = feature_[ ['src_ip', 'dst_ip', 'src_port', 'dst_port', 'Action']]

  scaler = StandardScaler()

  scaler.fit(feature)
  feature_trans = scaler.transform(feature)
  model = DBSCAN(eps=0.5, min_samples=5)
  predict = pd.DataFrame(model.fit_predict(feature_trans))
  predict.columns = ['predict']
  r = pd.concat([feature,predict],axis=1)
  
  # core_samples_mask = np.zeros_like(model.labels_, dtype=bool)
  # core_samples_mask[model.core_sample_indices_] = True
  labels = model.labels_

  n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
  n_noise_ = list(labels).count(-1)

  print("Dataset Size : ",len(feature))
  print("Estimated number of clusters: %d" % n_clusters_)
  print("Estimated number of noise points: %d" % n_noise_)
  return r, labels

In [20]:
def pair_plot(r):
  sns.pairplot(r,hue='predict')
  plt.show()


In [ ]:
# window size 만큼의의 데이터 추출
dbscan_result = dbscan(dataset)
pair_plot(dbscan_result)


In [ ]:
anomaly_data = dbscan_result[dbscan_result['predict']==-1]

In [ ]:
anomaly_data.to_csv("/content/drive/MyDrive/dbscan_anomaly_data.csv")